In [46]:
import pandas as pd
import urllib3
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import numpy as np

URL of the wiki website

In [47]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


Get the data from the website to a data frame

In [48]:
respond = requests.get(url)
soup = BeautifulSoup(respond.content, "html.parser")  # lay tao bo du lieu tu url
table = soup.find('table')  # Tim table
rows = table.findAll('tr')

df = pd.DataFrame()
cot0 = []
cot1 = []
cot2 = []

for row in rows[1:]: # Tim tung hang
    cells = row.findAll('td')
    cot0.append(cells[0].find(text=True))
    cot1.append(cells[1].find(text=True))
    cot2.append(cells[2].find(text=True))
df['PostalCode']=cot0
df['Borough']=cot1
df['Neighborhood']=cot2
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove 'Not assigned' cell in Borough columns, and set the 'Not assigned' cell in Neighborhood column to the value of Borough columns

In [49]:
df = df[df['Borough'] != 'Not assigned']
for i in range(len(df['Neighborhood'])):
    if df.iloc[i]['Neighborhood'] == 'Not assigned':
        df.iloc[i]['Neighborhood'] = df.iloc[i]['Borough']
df = df.reset_index()
df = df.drop(columns=['index'])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [50]:
dfgb = df.groupby('PostalCode').count()

Group the dataframe

In [51]:
postal_list = dfgb.index.values.tolist()

In [52]:
c=[]
for postal in postal_list:
    for i in range(len(df)):
        if df.iloc[i]['PostalCode'] == postal:
            a= df.iloc[i]['Borough']
    c.append(a)

In [53]:
b=[]
for postal in postal_list:
    a=''
    for i in range(len(df)):
        if df.iloc[i]['PostalCode'] == postal:
            
            a += '-' + df.iloc[i]['Neighborhood'].strip('\n')
    b.append(a)


In [54]:
d=[]
for i in b:
    a = i.split('-')
    e= a[1]
    if len(a) >2:
        for x in range(2,len(a)):
            e+= ', ' + a[x]
    d.append(e)


In [55]:
dfn = pd.DataFrame()
dfn['PostalCode'] = postal_list
dfn['Borough']=c
dfn['Neighborhood'] = d

print (dfn[:15])

   PostalCode      Borough                                       Neighborhood
0         M1B  Scarborough                                     Rouge, Malvern
1         M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
2         M1E  Scarborough                  Guildwood, Morningside, West Hill
3         M1G  Scarborough                                             Woburn
4         M1H  Scarborough                                          Cedarbrae
5         M1J  Scarborough                                Scarborough Village
6         M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park
7         M1L  Scarborough                    Clairlea, Golden Mile, Oakridge
8         M1M  Scarborough    Cliffcrest, Cliffside, Scarborough Village West
9         M1N  Scarborough                        Birch Cliff, Cliffside West
10        M1P  Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
11        M1R  Scarborough                                  Mary

In [56]:
dfn.shape

(103, 3)

Import the geocode libary

In [57]:
!conda install -c conda-forge geocoder --yes 

Solving environment: done

# All requested packages already installed.



In [58]:
import geocoder

Get the latitude and longitude

In [ ]:
latitude = []
longitude = []

for y in dfn['PostalCode']:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(y))
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
dfn['Latitude'] = latitude
dfn['Longitude'] = longitude
dfn.head()

In [ ]:
dfn.shape

Choose the data related to Toronto

In [ ]:
df_to = dfn[dfn['Borough'].str.contains('Toronto')]
df_to.head()

In [ ]:
CLIENT_ID = 'WB2MMTXYA1ROXPDLSVX5XCJCFLEVOI1QEWQ2PGTFGSXSJS1Z' # your Foursquare ID
CLIENT_SECRET = 'Q143TFXFSJQGQ1B225UGAUITQ0P1A3UVLVUC15ARGTESUB5V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

K means clustering model

In [ ]:
from sklearn.cluster import KMeans

df_to_kmeans = df_to.drop(columns=['PostalCode','Borough','Neighborhood'])
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_to_kmeans)
df_to['label']=kmeans.labels_


In [ ]:
df_to.head()

Import folium libary to create the map

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Create the map

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.676357,-79.293031], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_to['Latitude'], df_to['Longitude'], df_to['Neighborhood'], df_to['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters